# Import Libraries

In [ ]:
# Data Analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# NN model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint

# Evaluation
from sklearn.metrics import confusion_matrix, classification_report


# Data Augmentation

In [ ]:
train_generator = ImageDataGenerator(rotation_range = 360,
                                     width_shift_range = 0.05,
                                     height_shift_range = 0.05,
                                     shear_range = 0.05,
                                     zoom_range = 0.05,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     brightness_range = [0.75, 1.25],
                                     rescale = 1./255,
                                     validation_split = 0.2)

In [ ]:
IMAGE_DIR = "../input/tire-texture-image-recognition/Tire Textures/"
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 64
SEED_NUMBER = 123

gen_args = dict(target_size = IMAGE_SIZE,
                color_mode = 'grayscale',
                batch_size = BATCH_SIZE,
                class_mode = 'binary',
                classes = {'normal': 0, 'cracked': 1},
                seed = SEED_NUMBER)

train_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + 'training_data',
                                        subset = 'training', shuffle = True, **gen_args)
validation_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + 'training_data',
                                        subset = 'validation', shuffle = True, **gen_args)

In [ ]:
test_generator = ImageDataGenerator(rescale = 1./255)
test_dataset = test_generator.flow_from_directory(directory = IMAGE_DIR + 'testing_data',
                                                 shuffle = False,
                                                 **gen_args)

# Visualize the Image

## Visualize Image in Batch

In [ ]:
mapping_class = {0: 'ok', 1:'cracked'}
mapping_class

In [ ]:
def visualizeImageBatch(dataset, title):
    images, labels = next(iter(dataset))
    images = images.reshape(BATCH_SIZE, *IMAGE_SIZE)
    fig, axes = plt.subplots(8,8, figsize=(16,16))
    
    for ax, img, label in zip(axes.flat, images, labels):
        ax.imshow(img, cmap = 'gray')
        ax.axis('off')
        ax.set_title(mapping_class[label], size = 20)
    
    plt.tight_layout()
    fig.suptitle(title, size = 30, y = 1.05, fontweight = 'bold')
    plt.show()
    
    return images

In [ ]:
train_images = visualizeImageBatch(train_dataset,
                                  'FIRST BATCH OF THE TRAINING IMAGES\n(WITH DATA AUGMENTATION)')

In [ ]:
test_images = visualizeImageBatch(test_dataset,
                                 'FIRST BATCH OF TEST IMAGES\n(WITHOUT DATA AUGMENTATION)')

## Visualize Detailed Image

In [ ]:
img = np.squeeze(train_images[4])[75:100, 75:100]

fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111)
ax.imshow(img,cmap = 'gray')
ax.axis('off')

w,h = img.shape
for x in range(w):
    for y in range(h):
        value = img[x][y]
        ax.annotate('{:.2f}'.format(value), xy = (y,x),
                   horizontalalignment = 'center',
                   verticalalignment = 'center',
                   color = 'white' if value < 0.4 else 'black')

# Training the Network

In [ ]:
model_cnn = Sequential(
[
    # First convolution layer
    Conv2D(filters = 128,
          kernel_size = 3,
          strides = 2,
          activation= 'relu',
          input_shape = IMAGE_SIZE + (1,)),
    
    # First pooling layer
    MaxPooling2D(pool_size = 2,
                strides = 2),
    
    # Second convolution layer
    Conv2D(filters = 64,
          kernel_size = 3,
          strides = 2,
          activation= 'relu'),
    
    # Second pooling layer
    MaxPooling2D(pool_size = 2,
                strides = 2),  

    
    # Flattening
    Flatten(),
    
    # Fully-connected layer
    Dense(64, activation = 'relu'),
    Dropout(rate = 0.2),
    
    Dense(32, activation = 'relu'),
    Dropout(rate = 0.2),
    
    Dense(16, activation = 'relu'),
    Dropout(rate = 0.2),
     
    Dense(1, activation = 'sigmoid')
])

model_cnn.summary()

# Compile the Model

In [ ]:
model_cnn.compile(optimizer = 'adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])

# Model Fitting

In [ ]:
checkpoint = ModelCheckpoint('model/cnn_tire_texture_inspection_model.hdf5',
                            verbose = 1,
                             save_best_only = True,
                             monitor = 'val_loss',
                             mode = 'min'
                            )
model_cnn.fit(train_dataset,
             validation_data = validation_dataset,
             batch_size = 16,
             epochs = 12,
             callbacks = [checkpoint],
             verbose = 1)

# Training Evalution

In [ ]:
plt.subplots(figsize = (8,6))
sns.lineplot(data = pd.DataFrame(model_cnn.history.history,
                                index = range(1,1+len(model_cnn.history.epoch))))
plt.title('TRAINING EVALUATION', fontweight = 'bold', fontsize =20)
plt.xlabel('Epochs')
plt.ylabel('Metrics')

plt.legend(labels = ['val loss', 'val accuracy', 'train loss', 'train accuracy'])
plt.show

# Testing on Unseen Images

In [ ]:
best_model = load_model('model/cnn_tire_texture_inspection_model.hdf5')

In [ ]:
y_pred_prob = best_model.predict(test_dataset)

In [ ]:
THRESHOLD = 0.5
y_pred_class = (y_pred_prob >= THRESHOLD).reshape(-1,)
y_true_class = test_dataset.classes[test_dataset.index_array]

pd.DataFrame(confusion_matrix(y_true_class, y_pred_class),
             index = [['Actual', 'Actual'],['ok','cracked']],
             columns = [['Predicted', 'Predicted'],['ok','cracked']])

In [ ]:
print(classification_report(y_true_class, y_pred_class,digits = 4))